In [1]:
import pandas as pd
import numpy as np
import ast
from tqdm import tqdm_notebook as tqdm
import re

In [2]:
data = pd.read_csv('data/rap_corpus.csv', sep=';')
data.shape

(5419, 2)

In [3]:
data.head()

,title,text
0,Паук - Мой бит,"['Припев (4 раза):', 'Мой бит качает.', 'Он ни..."
1,Стриж - Алена,"['Ухоженная девочка', 'Брильянтовые стразы', '..."
2,Скато (Skato) - Хроника,"['Кровоточат раны, швы наложены на прошлое.', ..."
3,Bad Balance - Африка,['- Как уже сообщалось: в минувшую субботу поз...
4,"Триада - Записки сумасшедшего (feat. СВО, Нига...","['Когда-нибудь, касаясь взглядами,', 'Пройдем ..."


In [4]:
data.tail()

,title,text
5414,"Dessar - Постой, не уходи","['Я тушу боль последнею сигаретой ', 'В глазах..."
5415,Big Black Boots - Нам хорошо (feat. Тэона),"['Вот и мы, а где были вы', 'Когда мы начинали..."
5416,Каста - Гончая,"['На спидометре сто двадцать- мчим,', 'Трек по..."
5417,Krec - В никуда,"['Взять чемодан и умчать в никуда, ', 'Может, ..."
5418,Schokk - Грустная песня (feat. Maxat),"['Я иногда, будто сквозь время вижу в степи ру..."


In [5]:
## преобразование текста в список
rappers = []
texts = []
for i in tqdm(range(data.shape[0])):
    tmp = data.iloc[i,:]
    rappers.append(tmp.title.split(' - ')[0])
    txt = ast.literal_eval(tmp.text)
    txt = [x.lower() for x in txt]
    texts.append(txt)

In [6]:
print(len(rappers))
print(len(texts))

5420
5420


In [7]:
texts[1][:10]

['припев (4 раза):',
 'мой бит качает.',
 'он никого не напрягает.',
 '<w>',
 'мой микрофон включён, мой ди-джей на месте.',
 'приготовьтесь к уроку и все вместе',
 'поднимите руки вверх, когда я толкаю тему.',
 'теорему о том, как атаковать по биту смело.',
 'мои рифмы оставляют шрамы на сердцах.',
 'наводят страх, я вижу, ты в моих руках.']

In [8]:
def string_processing(string):
    symbols = ['%', '&', '(', ')', '*', '+', '/', '[', '\\', ']', '`',
               '{', '}', '~', '«', '»','’','“','”','•','…','№','\ufeff', '\xa0', '=', '\xad', '°',
               '^', '!', '?', ',', '...', '..', ':', ';', '<', '>', '"', '‘', '#']
    for s in symbols:
        string = string.replace(s, '')
    
    string = string.replace('$', 's')
    string = string.replace('ü', 'u')
    string = string.replace('є', 'е')
#     string = string.replace('і', 'i')
#     string = string.replace('ї', 'i')
#     string = string.replace('ң', 'н')
#     string = string.replace('à', 'а')
#     string = string.replace('ß', 'b')
#     string = string.replace('ó', 'о')
    string = string.replace('é', 'e')
    
#     string = string.replace('\t', '')

    string = string.replace('–', '-')
    string = string.replace('—', '-')
    string = string.replace('_', '-')
    string = string.replace(' - ', ' ')
    string = string.replace(' -', ' ')
    string = string.replace('- ', ' ')
    
    if string.startswith('-'):
        string = string[1:]

    if string.endswith('-'):
        string = string[:-1]
    
    string = string.replace('.', ' ')
    
    string = string.replace('@', ' ')
    string = string.replace("''", '')
    string = string.replace(" '", ' ')

    
    string = string.replace('   ', ' ')
    string = string.replace('  ', ' ')
    
    
    string = string.strip()
        
    re.sub('[ ]{2,}', ' ', string)
    
    return string

In [9]:
txt

['я иногда, будто сквозь время вижу в степи руины,',
 'покрытые пеплом, весь мир и шарон лишь серой пустыни',
 'тошно смотреть телевизор, толпы детей и дебилов',
 'кто-то наденьте очки им, иначе дверь не увидят',
 'я не верил в религии, они потеряны в книгах',
 'а ватикан время дворец педофилов',
 'конец уже близок, черный опездал не лучше',
 'тот же отец или сын, жирный шит всем на уши',
 'лет через сто мы с бородой и узкоглазые будем',
 'ведь пол европы сегодня уж шоколадные, хули',
 'эту тему не любят, а наши дети толкуют',
 'во всем я больше чем уверен, блять, евреев засудят',
 'легче быть кем угодно, кумиров таск голливуд',
 'ценности те, что берегли нам предки, нас не ебут',
 'экран, радио, журналы, он-лайн ',
 'нас ебут в глаза и уши, как шалаву в анал...',
 'дохнут от рака, мы курим, дохнут от спида-ебёмся,',
 'но нам втирают про озоны - мы боимся солнца?',
 'выборы толстожопых пидоров с мордой, шире калитки',
 'и похуй, какая разница, кто пиздить будет, они или кто то?',
 'а м

In [10]:
from nltk import WhitespaceTokenizer

In [11]:
## чистка
texts_ = []
rappers_ = []
lens = []
i = 0
for txt in tqdm(texts):
    rapper = rappers[i]
    i += 1

    if len(txt) < 5:
        continue
    
    tmp = []    
    for string in txt:
        
        if len(string) <= 10:
            continue
        if string.startswith('[') and string.endswith(']'):
            continue
        if string.startswith('(') and string.endswith(')'):
            continue
            
        if string.count('припев') > 0:
            continue
        if string.count('куплет') > 0:
            continue
        if string.count('интро') > 0:
            continue

        string = string_processing(string)
        
        if string == '':
            continue
        
        string = string + ' <end> '
        
#         if re.match('^[0-9][.: ]', string):
#             continue
#         re.sub('[ ]{2,}', ' ', string)
#         re.sub('[ ]{2,}', ' ', string)
#         re.sub('-{2,}', '', string)
        
#         srtring = string.replace('-4', ' 4')
#         srtring = string.replace('-и', ' и')
#         srtring = string.replace('-ов', ' ов')
#         srtring = string.replace('-особа', ' особа')
#         srtring = string.replace('-привет', ' привет')
              
        lens.append(len(WhitespaceTokenizer().tokenize(string)))    
        tmp.append(string)

    texts_.append(tmp)
    rappers_.append(rapper)


print(len(texts_))
print(len(rappers_))                


5372
5372


In [12]:
print(np.median(lens))
print(np.mean(lens))

7.0
7.352350498508214


In [13]:
texts_[30:40]

[['из константы <end> ',
  'ааа ха из константы здесь <end> ',
  'из той самой <end> ',
  'это русский стандарт <end> ',
  'спокойно норд-ист а не смольный <end> ',
  'не первопрестольная кварталам застроенным <end> ',
  'где дтп все подстроены людям достойным <end> ',
  'я сказал людям достойным <end> ',
  'спокойно норд-ист а не смольный <end> ',
  'не первопрестольная кварталам застроенным <end> ',
  'где дтп все подстроены людям достойным <end> ',
  'я сказал людям достойным <end> ',
  'жаренный вспыльчивый <end> ',
  'там же где не один косяк вылечили <end> ',
  'сидели увеличивали <end> ',
  'здесь дедушка мороз за снег берёт наличными <end> ',
  'если не безразлично то номера в личке <end> ',
  'так чисто попудрить личико <end> ',
  'не одноклассники не отличники <end> ',
  'таз гостинцами напичканный <end> ',
  'вечно мы хотим поднять как с ипотечного <end> ',
  'без аптечки таз <end> ',
  'так что стелите аккуратненько и крутите плотненько <end> ',
  'сто грамм за наших плотни

In [14]:
all_text = ''
for txt in texts_:
    all_text += ' '.join(txt) 

In [15]:
len(all_text)

12596803

In [16]:
words = WhitespaceTokenizer().tokenize(all_text)
print(len(words))
words = sorted(set(words))

2163635


In [17]:
words[5000:]

['алкогольную',
 'алкогольный',
 'алкоголю',
 'алкоголя',
 'алкокапли',
 'алколига',
 'алколойд',
 'алкомаркете',
 'алкоритмы',
 'алкот',
 'алкота',
 'алкотестер',
 'алкофанка',
 'алкоэнерджайзера',
 'алкояд',
 'алла',
 'алладин',
 'алладина',
 'аллах',
 'аллаха',
 'аллахом',
 'аллаху',
 'алле',
 'аллегории',
 'аллегорий',
 'аллегорию',
 'аллегория',
 'аллегрова',
 'аллегрову',
 'аллее',
 'аллеи',
 'аллей',
 'аллейкум',
 'аллен',
 'аллергию',
 'аллергия',
 'аллею',
 'аллея',
 'аллеям',
 'аллеями',
 'аллеях',
 'аллигатора',
 'аллигаторов',
 'аллигаторы',
 'аллии',
 'аллилуйя',
 'аллилуйях',
 'алло',
 'аллой',
 'аллюминиевую',
 'аллюминиевыми',
 'аллюминий',
 'алля',
 'аллё',
 'алма-ата',
 'алма-аты',
 'алмаатинский',
 'алмааты',
 'алмаз',
 'алмаза',
 'алмазам',
 'алмазами',
 'алмазах',
 'алмазная',
 'алмазов',
 'алмазу',
 'алмазы',
 'алматы',
 'ало',
 'алого',
 'алой',
 'алом',
 'алочные',
 'алоэ',
 'алсу',
 'алсю',
 'алтае',
 'алтаре',
 'алтари',
 'алтарь',
 'алтарю',
 'алтарях',
 'алт

In [18]:
char = sorted(set(all_text))

In [19]:
char

[' ',
 "'",
 '-',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '<',
 '>',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'а',
 'б',
 'в',
 'г',
 'д',
 'е',
 'ж',
 'з',
 'и',
 'й',
 'к',
 'л',
 'м',
 'н',
 'о',
 'п',
 'р',
 'с',
 'т',
 'у',
 'ф',
 'х',
 'ц',
 'ч',
 'ш',
 'щ',
 'ъ',
 'ы',
 'ь',
 'э',
 'ю',
 'я',
 'ё']

In [20]:
d = {'rapper':rappers_, 'text':texts_}
df = pd.DataFrame(d)
df.head()

,rapper,text
0,Многоточие (DOTSFAM),"[dots family <end> , dots family <end> , рэп э..."
1,Паук,"[мой бит качает <end> , он никого не напрягает..."
2,Стриж,"[ухоженная девочка <end> , брильянтовые стразы..."
3,Скато (Skato),[кровоточат раны швы наложены на прошлое <end>...
4,Bad Balance,[как уже сообщалось в минувшую субботу поздно ...


In [21]:
df.head(200)

,rapper,text
0,Многоточие (DOTSFAM),"[dots family <end> , dots family <end> , рэп э..."
1,Паук,"[мой бит качает <end> , он никого не напрягает..."
2,Стриж,"[ухоженная девочка <end> , брильянтовые стразы..."
3,Скато (Skato),[кровоточат раны швы наложены на прошлое <end>...
4,Bad Balance,[как уже сообщалось в минувшую субботу поздно ...
5,Триада,"[когда-нибудь касаясь взглядами <end> , пройде..."
6,Дэфолиант,"[ты говоришь во вселенной мы одни <end> , но о..."
7,Maestro A-Sid,"[и понимаем что нам не быть вдвоем <end> , отн..."
8,V-Style,"[когда-нибудь эти люди станут умнее <end> , не..."
9,Fist,"[а напоследок я бухну ха-ха-ха <end> , солнце ..."


In [22]:
df.to_csv('data/rap.csv', sep=';', index=False)